In [7]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm.notebook import tqdm
import pickle
import logging
import os
import requests
import dotenv
from IPython.display import display, HTML, IFrame
import json

import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network

dotenv.load_dotenv()

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class KnowledgeGraph:
    def __init__(self, similarity_threshold=0.8, base_url=None, api_key=None):
        self.graph = nx.Graph()
        self.similarity_threshold = similarity_threshold
        self.df = None
        self.base_url = base_url
        self.headers = {"Authorization": f"Bearer {api_key}"} if api_key else {}

    def load_data(self, file_path):
        logging.info("Chargement des données...")
        self.df = pd.read_csv(file_path, sep='\t')
        self.df['content'] = self.df['content'].astype(str).replace('nan', '')
        self.df['head'] = self.df['head'].astype(str).replace('nan', '')
        self.df = self.df[self.df['content'].str.strip() != '']
        logging.info(f"Données chargées. Nombre d'articles : {len(self.df)}")

    def create_embeddings(self):
        logging.info("Création des embeddings...")
        tqdm.pandas()  # Ceci active la barre de progression pour pandas
        self.df['embedding'] = self.df['content'].progress_apply(lambda x: self.get_embedding(x))
        logging.info("Embeddings créés.")

    def get_embedding(self, text):
        url = f"{self.base_url}/ollama/api/embeddings"
        payload = {
            "model": "mxbai-embed-large",
            "prompt": text
        }
        response = requests.post(url, json=payload, headers=self.headers)
        if response.status_code == 200:
            return response.json()['embedding']
        else:
            raise Exception(f"Erreur lors de la génération de l'embedding: {response.text}")

    def build_graph(self):
        logging.info("Construction du graphe...")
        for _, row in tqdm(self.df.iterrows(), total=len(self.df), desc="Ajout des nœuds"):
            node_id = str(row['id_enccre'])  # Conversion de l'ID en chaîne de caractères
            self.graph.add_node(node_id, **row.to_dict())

        embeddings = np.array(self.df['embedding'].tolist())
        similarity_matrix = cosine_similarity(embeddings)

        for i in tqdm(range(len(self.df)), desc="Ajout des arêtes"):
            for j in range(i+1, len(self.df)):
                if similarity_matrix[i][j] > self.similarity_threshold:
                    self.graph.add_edge(str(self.df.iloc[i]['id_enccre']), str(self.df.iloc[j]['id_enccre']), weight=similarity_matrix[i][j])

        logging.info(f"Graphe construit. Nombre de nœuds : {self.graph.number_of_nodes()}, Nombre d'arêtes : {self.graph.number_of_edges()}")

    def save_graph(self, file_path):
        with open(file_path, 'wb') as f:
            pickle.dump(self, f)
        logging.info(f"Graphe sauvegardé dans {file_path}")

    @classmethod
    def load_graph(cls, file_path):
        with open(file_path, 'rb') as f:
            return pickle.load(f)

    def get_node_info(self, node_id):
        return self.graph.nodes[node_id]

    def get_neighbors(self, node_id):
        return list(self.graph.neighbors(node_id))

    def get_subgraph(self, node_ids):
        return self.graph.subgraph(node_ids)

    def get_most_connected_nodes(self, n=10):
        return sorted(self.graph.degree, key=lambda x: x[1], reverse=True)[:n]

    def get_shortest_path(self, start_node, end_node):
        return nx.shortest_path(self.graph, start_node, end_node)

    def get_connected_components(self):
        return list(nx.connected_components(self.graph))

    
    def visualize_graph(self, output_file='graph.html', height='500px', width='100%'):
        """
        Visualise le graphe de connaissances interactivement.
        
        :param output_file: Nom du fichier HTML de sortie
        :param height: Hauteur du graphe (en pixels ou pourcentage)
        :param width: Largeur du graphe (en pixels ou pourcentage)
        """
        nt = Network(height=height, width=width, notebook=True)
        
        # Ajout des nœuds
        for node in self.graph.nodes():
            node_id = str(node)  # Conversion de l'ID en chaîne de caractères
            nt.add_node(node_id, label=self.graph.nodes[node].get('title', node_id))
        
        # Ajout des arêtes
        for edge in self.graph.edges():
            nt.add_edge(str(edge[0]), str(edge[1]))  # Conversion des IDs en chaînes de caractères
        
        # Sauvegarde et affichage
        nt.save_graph(output_file)
        return nt

        
    def visualize_subgraph(self, node_ids, output_file='subgraph.html', height=500, width=700):
        subgraph = self.get_subgraph(node_ids)
        nt = Network(notebook=True, cdn_resources='remote', height=f"{height}px", width=f"{width}px")
        
        for node in subgraph.nodes():
            node_id = str(node)
            node_data = subgraph.nodes[node]
            label = node_data.get('head', node_id)  # Utiliser 'head' comme étiquette
            title = f"ID: {node_id}<br>Contenu: {node_data.get('content', '')[:100]}..."  # Contenu affiché au survol
            nt.add_node(node_id, label=label, title=title)
        
        for edge in subgraph.edges():
            weight = subgraph[edge[0]][edge[1]].get('weight', 1)
            nt.add_edge(str(edge[0]), str(edge[1]), value=weight)
        
        nt.set_options("""
        var options = {
            "nodes": {
                "font": {
                    "size": 12
                }
            },
            "edges": {
                "color": {
                    "inherit": true
                },
                "smooth": false
            },
            "physics": {
                "forceAtlas2Based": {
                    "gravitationalConstant": -26,
                    "centralGravity": 0.005,
                    "springLength": 230,
                    "springConstant": 0.18
                },
                "maxVelocity": 146,
                "solver": "forceAtlas2Based",
                "timestep": 0.35,
                "stabilization": {
                    "enabled": true,
                    "iterations": 1000,
                    "updateInterval": 25
                }
            }
        }
        """)
        
        # Sauvegarder le graphe
        nt.save_graph(output_file)
        
        # Obtenir le chemin absolu du fichier
        abs_file_path = os.path.abspath(output_file)
        
        # Afficher le graphe en utilisant IFrame
        return IFrame(src=f'file://{abs_file_path}', width=width, height=height)

class QueryEngine:
    def __init__(self, knowledge_graph, base_url, api_key):
        self.knowledge_graph = knowledge_graph
        self.base_url = base_url
        self.headers = {"Authorization": f"Bearer {api_key}"}

    def get_embedding(self, text):
        url = f"{self.base_url}/ollama/api/embeddings"
        payload = {
            "model": "mxbai-embed-large",
            "prompt": text
        }
        response = requests.post(url, json=payload, headers=self.headers)
        if response.status_code == 200:
            return response.json()['embedding']
        else:
            raise Exception(f"Erreur lors de la génération de l'embedding: {response.text}")

    def generate_response(self, prompt):
        url = f"{self.base_url}/ollama/api/generate"
        payload = {
            "model": "llama3.1:8b-instruct-q4_0",
            "prompt": f"Voici une question d'un utiisateur : {prompt}. Réponds en t'aidant uniquement du contenu qui t'es fourni."
        }
        response = requests.post(url, json=payload, headers=self.headers, stream=True)
        if response.status_code == 200:
            full_response = ""
            for line in response.iter_lines():
                if line:
                    json_response = json.loads(line)
                    full_response += json_response.get('response', '')
                    if json_response.get('done', False):
                        break
            return full_response
        else:
            raise Exception(f"Erreur lors de la génération de la réponse: {response.text}")

    def query(self, query: str):
        logging.info("Traitement de la requête...")
        try:
            embedding = self.get_embedding(query)
            
            # Trouver les nœuds les plus similaires
            similarities = []
            for node in self.knowledge_graph.graph.nodes():
                node_embedding = self.knowledge_graph.graph.nodes[node]['embedding']
                similarity = cosine_similarity([embedding], [node_embedding])[0][0]
                similarities.append((node, similarity))
            
            # Trier par similarité décroissante et prendre les 5 premiers
            most_similar = sorted(similarities, key=lambda x: x[1], reverse=True)[:5]
            
            context = ""
            for node, _ in most_similar:
                context += f"\n{self.knowledge_graph.graph.nodes[node]['content']}"
                # Ajouter le contenu des nœuds voisins
                for neighbor in self.knowledge_graph.get_neighbors(node):
                    context += f"\n{self.knowledge_graph.graph.nodes[neighbor]['content']}"
            
            prompt = f"Using this context: {context}\nRespond to this query: {query}"
            response = self.generate_response(prompt)

            return response, [node for node, _ in most_similar], [self.knowledge_graph.graph.nodes[node]['content'] for node, _ in most_similar]
        except Exception as e:
            logging.error(f"Erreur lors du traitement de la requête : {str(e)}")
            logging.error(f"Type d'erreur : {type(e).__name__}")
            logging.error(f"Détails de l'erreur : {e.args}")
            return "Désolé, je n'ai pas pu traiter votre requête.", [], []

class Chatbot:
    def __init__(self, data_file, base_url, api_key):
        self.data_file = data_file
        self.base_url = base_url
        self.api_key = api_key
        self.knowledge_graph = None
        self.query_engine = None

    def initialize(self):
        graph_file = "knowledge_graph.pkl"
        if os.path.exists(graph_file):
            logging.info("Chargement du graphe existant...")
            self.knowledge_graph = KnowledgeGraph.load_graph(graph_file)
        else:
            logging.info("Création d'un nouveau graphe...")
            self.knowledge_graph = KnowledgeGraph(base_url=self.base_url, api_key=self.api_key)
            self.knowledge_graph.load_data(self.data_file)
            self.knowledge_graph.create_embeddings()
            self.knowledge_graph.build_graph()
            self.knowledge_graph.save_graph(graph_file)

        self.query_engine = QueryEngine(self.knowledge_graph, self.base_url, self.api_key)

    def chat(self, user_query):
        try:
            response, doc_ids, doc_contents = self.query_engine.query(user_query)
            
            print(f"Réponse : {response}\n")
            print("Sources pertinentes :")
            for i, (doc_id, content) in enumerate(zip(doc_ids, doc_contents), 1):
                print(f"{i}. ID: {doc_id}")
                print(f"   Contenu: {content[:100]}...\n")
            
            # Visualisation du sous-graphe
            print("Visualisation du sous-graphe :")
            subgraph_viz = self.knowledge_graph.visualize_subgraph(doc_ids, output_file='query_subgraph.html')
            display(subgraph_viz)
        except Exception as e:
            print(f"Erreur lors du traitement de la requête : {str(e)}")

# Initialisation du chatbot
BASE_URL = "https://lemum.duckdns.org"
API_KEY = os.getenv("API_KEY")
chatbot = Chatbot("data/EDdA_dataframe_withContent_test.tsv", BASE_URL, API_KEY)
chatbot.initialize()

2024-09-03 16:52:29,853 - INFO - Chargement du graphe existant...


In [8]:
# Exemple d'utilisation
chatbot.chat("Donne moi des exemples récents d'abdication.")

2024-09-03 16:52:39,357 - INFO - Traitement de la requête...


Réponse : Voici quelques exemples récents d'abdication que j'ai trouvés dans le texte :

* Dioclétien a abdiqué la Couronne.
* Charles V. a abdiqué la Couronne.
* Le Parlement d'Angleterre a décidé que la violation des Lois faite par le Roi Jacques, en quittant son Royaume sans avoir pourvû à l'administration nécessaire des affaires pendant son absence, emportoit avec elle l'abdication de la Couronne.

Notez également que l'exemple du roi Philippe IV d'Espagne qui a "résigné" la Couronne est mentionné dans le texte, mais cela constitue une différence avec l'abdication, car il s'est fait en faveur d'une personne tierce.

Sources pertinentes :
1. ID: v1-95-3
   Contenu: Abdication, au Palais, est aussi quelquefois synonyme 
à abandonnement. V. Abandonnement.
(H)...

2. ID: v1-95-2
   Contenu: Abdication s'est dit encore de l'action d'un homme 
libre qui renonçoit à sa liberté, & se faisoit v...

3. ID: v1-95-0
   Contenu: ABDICATION, s. f. acte par lequel un Magistrat
ou une personne en 